<a href="https://colab.research.google.com/github/cedamusk/earthquakes-and-Python/blob/main/earthquake_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy basemap requests beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.0/936.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [ ]:
import requests #for making HTTP requests to web servers
from bs4 import BeautifulSoup #for parsing HTML and XML documents
import pandas as pd#For dat manipulation and analysis
import datetime#for manipulating dates and times
from sklearn.model_selection import GridSearchCV#For hyperparameter tuning in machine learning models
from imblearn.pipeline import Pipeline#for constructing a machine learning workflow with steps for data preprocessing and modelling
from imblearn.over_sampling import SMOTE#for handling imbalanced datasets by oversampling the minority class
from sklearn.model_selection import train_test_split#for splitting the dataset into training and testing sets
from sklearn.ensemble import RandomForestClassifier#for building the earthquake prediction model
from sklearn.metrics import classification_report, accuracy_score#for evaluating the perfomance of the machine learning model
import seaborn as sns#for creating attractive and informative statistical graphics
import matplotlib.pyplot as plt#for plotting graphs and charts
import folium#for creating interactive maps
from folium.plugins import MarkerCluster#for clustering earthquake locations on the map for better readability

In [ ]:
def scrape_earthquake_data():
  #this line defines a function
  url='https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv'
  #this line defines the variable 'url' and assigns the url of a csv file containing earthquake data form the USGS website
  response=requests.get(url)
  #this line uses the requests library ro send a GET request to the specified url
  if response.status_code==200:
    #this line checks the status code of the response. A status code of 200 indicates a succesful request
    data=pd.read_csv(url)
    #if the status code is 200, this line uses the pandas library to read the CSV data from the URL into a Pandas DataFrame object named data
    return data
    #if succesful this line returns the 'data' DataFrame from the function
  else:
    print(f"Failed to retrieve dta. Status code:{response.status_code}")
    return None
    #if the status code is not 200, and error message is printed


    earthquake_data=scrape_earthquake_data()


    if earthquake_data is not None:
      print(earthquake_data.head())
    else:
      print("No earthquake data retrieved")
      #Displays the first few rows of the dataset

In [ ]:

import pandas as pd

def preprocess_data(df):
    df = df[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
    df.columns = ['Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude', 'Location']
    # Select relevant columns and rename them for consistency

    df['Time'] = pd.to_datetime(df['Time'])
    # Convert time to datetime

    df['Year'] = df['Time'].dt.year
    df['Month'] = df['Time'].dt.month
    df['Day'] = df['Time'].dt.day
    df['Hour'] = df['Time'].dt.hour
    df['Minute'] = df['Time'].dt.minute
    df['Second'] = df['Time'].dt.second
    # Extract additional features from the datetime column

    df = df.dropna()
    # Handle missing values if any

    return df
    #return the processed DataFrame


earthquake_data = scrape_earthquake_data()
if earthquake_data is not None:
    processed_data = preprocess_data(earthquake_data)
    print(processed_data.head())
else:
    print("No earthquake data retrieved")
    #applying the preprocessing function and displaying the data

NameError: name 'scrape_earthquake_data' is not defined

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#importing required libraries

def plot_magnitude_distribution(df):
    plt.figure(figsize=(14, 8))
    #setting up the plot

    sns.histplot(df['Magnitude'], bins=30, color='cornflowerblue', edgecolor='black', linewidth=1.2)
    # Create the histogram with KDE plot using blue hues

    mean_magnitude = df['Magnitude'].mean()
    plt.axvline(mean_magnitude, color='red', linestyle='--', linewidth=2)
    plt.text(mean_magnitude + 0.1, plt.ylim()[1] * 0.8, f'Mean: {mean_magnitude:.2f}', color='red', fontsize=14, fontweight='bold')
    # Add a vertical line at the mean magnitude

    median_magnitude = df['Magnitude'].median()
    plt.axvline(median_magnitude, color='green', linestyle='--', linewidth=2)
    plt.text(median_magnitude + 0.1, plt.ylim()[1] * 0.8, f'Median: {median_magnitude:.2f}', color='green', fontsize=14, fontweight='bold')
    # Add a vertical line at the median magnitude

    plt.title('Distribution of Earthquake Magnitudes', fontsize=20, fontweight='bold', color='navy')
    plt.xlabel('Magnitude', fontsize=16, fontweight='bold')
    plt.ylabel('Frequency', fontsize=16, fontweight='bold')
    # Customizing the title and labels

    plt.xticks(fontsize=12, fontweight='bold')
    plt.yticks(fontsize=12, fontweight='bold')
    # Customize the tick parameters

    plt.grid(True, linestyle='--', alpha=0.6)
    # Add custom grid

    plt.axvspan(5, 7, color='red', alpha=0.3, lw=0)
    plt.text(6, plt.ylim()[1] * 0.95, 'Critical Range (5-7)', color='navy', fontsize=14, fontweight='bold', ha='center')
    # Highlight a critical magnitude range

    max_freq_bin = df['Magnitude'].value_counts().idxmax()
    max_freq_count = df['Magnitude'].value_counts().max()
    plt.annotate(f'Highest Frequency\n{max_freq_bin} Mag: {max_freq_count} times',
                 xy=(max_freq_bin, df['Magnitude'].value_counts()[max_freq_bin]),
                 xytext=(max_freq_bin + 0.5, df['Magnitude'].value_counts()[max_freq_bin] + 5),
                 arrowprops=dict(facecolor='black', shrink=0.05),
                 fontsize=14, fontweight='bold',
                 bbox=dict(boxstyle="round, pad=0.3", edgecolor='black', facecolor="lightgray", alpha=0.8))
    # Adding annotations for the highest frequency bin

    plt.show()
    # Display the plot
    plot_magnitude_distribution(processed_data)


In [ ]:
import folium
from folium.plugins import MarkerCluster
#Importing required Libaries
def plot_enhanced_geo_map(df):
    m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=2)
    # Initialize the map centered around the mean coordinates

    marker_cluster = MarkerCluster().add_to(m)
    # Create a marker cluster

    for _, row in df.iterrows():
      #Looping through the DataFrame to Add markers
        magnitude = row['Magnitude']
        if magnitude < 3.0:
            color = 'blue'
        elif 3.0 <= magnitude < 5.0:
            color = 'green'
        elif 5.0 <= magnitude < 7.0:
            color = 'orange'
        else:
            color = 'red'
        #Determining Marker Color based on Magnitude

        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=5,
            popup=(
                f"Location: {row['Location']}<br>"
                f"Time: {row['Time']}<br>"
                f"Magnitude: {row['Magnitude']}<br>"
                f"Depth: {row['Depth']} km"
            ),
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(marker_cluster)
        #adding Markers to the map

    return m
    #returning the map


enhanced_earthquake_map = plot_enhanced_geo_map(processed_data)
enhanced_earthquake_map.save('enhanced_earthquake_map.html')
enhanced_earthquake_map


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
#importing required libraries

def build_and_evaluate_model_with_smote(df):
    features = ['Latitude', 'Longitude', 'Depth', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']
    target = 'Magnitude'
    #selecting features and target variable

    df['Target'] = (df['Magnitude'] >= 5.0).astype(int)
    # Create the target variable

    X = df[features]
    y = df['Target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #splitting the data into Training and Testing sets

    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    #handling class imbalance with SMOTE

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_res, y_train_res)
    #training the random forest model

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    #making predictions and evaluating the model

    return accuracy, report, y_test, y_pred
    #returning the results


accuracy, report, y_test, y_pred = build_and_evaluate_model_with_smote(processed_data)
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
